In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from data_utils import get_abs_path, split_dataset
from data_utils import parse_annotations_xml, create_annotations_list

In [ ]:
root_dir = get_abs_path(1)
dataset_images_dir = root_dir / 'data' / 'images'
dataset_annotations_dir = root_dir / 'data' / 'annotations'
train_images_dir = root_dir / 'road_signs_detection' / 'train' / 'images'
test_images_dir = root_dir / 'road_signs_detection' / 'test' / 'images'
train_annotations_dir = root_dir / 'road_signs_detection' / 'train' / 'annotations'
test_annotations_dir = root_dir / 'road_signs_detection' / 'test' / 'annotations'
train_images_dir.mkdir(exist_ok=True, parents=True)
test_images_dir.mkdir(exist_ok=True, parents=True)
train_annotations_dir.mkdir(exist_ok=True, parents=True)
test_annotations_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
# split_dataset(  dataset_images_dir, dataset_annotations_dir,
#                 train_images_dir, train_annotations_dir,
#                 test_images_dir, test_annotations_dir)

In [ ]:
train_annotations_list = create_annotations_list(train_annotations_dir)
train_df = pd.DataFrame(train_annotations_list)
print('\nTrain stats:')
print(train_df['class'].value_counts())

test_annotations_list = create_annotations_list(test_annotations_dir)
test_df = pd.DataFrame(test_annotations_list)
print('\nTest stats:')
print(test_df['class'].value_counts())

In [ ]:
class_idx = {'speedlimit': 0, 'stop': 1, 'crosswalk': 2, 'trafficlight': 3}
train_df['class_idx'] = train_df['class'].apply(lambda i: class_idx[i])

train_df.tail()